In [1]:
#This is the modified version of the ladder network code from https://github.com/rinuboney/ladder
#Certain modfications are made to use & experiment with gene expression data
#
import pickle
import numpy
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit

class DataSet(object):

  def __init__(self, dataset, labels):
    features = 20207
    num_labels = 60
    
    self._dataset = dataset
    self._labels = labels
    self._epochs_completed = 0
    self._index_in_epoch = 0
    self._num_examples = dataset.shape[0]

  @property
  def dataset(self):
    return self._dataset

  @property
  def labels(self):
    return self._labels

  @property
  def num_examples(self):
    return self._num_examples

  @property
  def epochs_completed(self):
    return self._epochs_completed

  def next_batch(self, batch_size):
    """Return the next `batch_size` examples from this data set."""
    #print ("index", self._index_in_epoch)
    start = self._index_in_epoch
    self._index_in_epoch += batch_size
    if self._index_in_epoch > self._num_examples:
      # Finished epoch
      self._epochs_completed += 1
      # Shuffle the data
      perm = numpy.arange(self._num_examples)
      numpy.random.shuffle(perm)
      self._dataset = self._dataset[perm]
      self._labels = self._labels[perm]
      # Start next epoch
      start = 0
      self._index_in_epoch = batch_size
      assert batch_size <= self._num_examples
    end = self._index_in_epoch
    return self._dataset[start:end], self._labels[start:end]

class SemiDataSet(object):
    def __init__(self, dataset, labels, n_labeled):
        
        self.n_labeled = n_labeled

        # Unlabled DataSet
        self.unlabeled_ds = DataSet(dataset, labels)

        # Labeled DataSet
        self.num_examples = self.unlabeled_ds.num_examples
        indices = numpy.arange(self.num_examples)
        shuffled_indices = numpy.random.permutation(indices)
        dataset = dataset[shuffled_indices]
        labels = labels[shuffled_indices]
        y = numpy.array([numpy.arange(2)[l==1][0] for l in labels])
        
        idx = indices[y==0][:5]
        n_classes = y.max() + 1
        n_from_each_class = n_labeled // n_classes
        i_labeled = []
        for c in range(n_classes):
            i = indices[y==c][:n_from_each_class]
            i_labeled += list(i)
        l_dataset = dataset[i_labeled]
        l_labels = labels[i_labeled]
        self.labeled_ds = DataSet(l_dataset, l_labels)

    def next_batch(self, batch_size):
        #print ("batch size semi", batch_size)
        unlabeled_dataset, _ = self.unlabeled_ds.next_batch(batch_size)
     
        if batch_size > self.n_labeled:
            labeled_dataset, labels = self.labeled_ds.next_batch(self.n_labeled)
        else:
            labeled_dataset, labels = self.labeled_ds.next_batch(batch_size)
            #print (labeled_dataset.shape)
        #print ("labels shape aasd", labels.shape)
        #print (labels)
        dataset = numpy.vstack([labeled_dataset, unlabeled_dataset])
        return dataset, labels

In [2]:
#one-hot label
def dense_to_one_hot(labels_dense, num_classes=2):

  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  print(num_labels)
  index_offset = numpy.arange(num_labels) * num_classes
  labels_one_hot = numpy.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot


In [3]:
#fix labels  1 for tumoral, 0 for healthy
def fix_label(labels):
    labels= [1 if x==1 else 0 for x in labels]
    
    return numpy.array(labels)


In [5]:
import tensorflow as tf
#import input_data
import math
import os
import csv
from tqdm import tqdm
import sklearn as sk
from sklearn.metrics import confusion_matrix


LOGDIR = "./log"

layer_sizes = [20207, 2000, 1000, 500, 250, 10,2]

L = len(layer_sizes) - 1  # number of layers



num_epochs = 100
num_labeled = 60
num_examples = 730

learning_rate = 0.01

batch_size = 60

num_iter = (num_examples//batch_size) * num_epochs  # number of loop iterations

inputs = tf.placeholder(tf.float32, shape=(None, layer_sizes[0]), name= "input")
outputs = tf.placeholder(tf.float32, name = "output")


def bi(inits, size, name):
    with tf.name_scope(name):
        b = tf.Variable(inits * tf.ones([size]), name="B")
        tf.summary.histogram("bias", b)
        return b

def wi(shape, name):
    with tf.name_scope(name):
        w = tf.Variable(tf.random_normal(shape, name="W")) / math.sqrt(shape[0])
        tf.summary.histogram("weight", w)
        return w

shapes = list(zip(list(layer_sizes)[:-1], list(layer_sizes[1:])))  # shapes of linear layers

weights = {'W': [wi(s, "W") for s in shapes],  # Encoder weights
           'V': [wi(s[::-1], "V") for s in shapes],  # Decoder weights
           # batch normalization parameter to shift the normalized value
           'beta': [bi(0.0, layer_sizes[l+1], "beta") for l in range(L)],
           # batch normalization parameter to scale the normalized value
           'gamma': [bi(1.0, layer_sizes[l+1], "beta") for l in range(L)]}

noise_std = 0.3  # scaling factor for noise used in corrupted encoder

# hyperparameters that denote the importance of each layer
denoising_cost = [1000.0, 10.0, 0.10, 0.10, 0.10, 0.10, 0.10]

join = lambda l, u: tf.concat([l, u], 0)
labeled = lambda x: tf.slice(x, [0, 0], [batch_size, -1]) if x is not None else x
unlabeled = lambda x: tf.slice(x, [batch_size, 0], [-1, -1]) if x is not None else x
split_lu = lambda x: (labeled(x), unlabeled(x))

training = tf.placeholder(tf.bool)

ewma = tf.train.ExponentialMovingAverage(decay=0.99)  # to calculate the moving averages of mean and variance
bn_assigns = []  # this list stores the updates to be made to average mean and variance


def batch_normalization(batch, mean=None, var=None):
    if mean is None or var is None:
        mean, var = tf.nn.moments(batch, axes=[0])
    return (batch - mean) / tf.sqrt(var + tf.constant(1e-10))

# average mean and variance of all layers
running_mean = [tf.Variable(tf.constant(0.0, shape=[l]), trainable=False) for l in layer_sizes[1:]]
running_var = [tf.Variable(tf.constant(1.0, shape=[l]), trainable=False) for l in layer_sizes[1:]]


def update_batch_normalization(batch, l):
    "batch normalize + update average mean and variance of layer l"
    mean, var = tf.nn.moments(batch, axes=[0])
    assign_mean = running_mean[l-1].assign(mean)
    assign_var = running_var[l-1].assign(var)
    bn_assigns.append(ewma.apply([running_mean[l-1], running_var[l-1]]))
    with tf.control_dependencies([assign_mean, assign_var]):
        return (batch - mean) / tf.sqrt(var + 1e-10)


def encoder(inputs, noise_std):
    h = inputs + tf.random_normal(tf.shape(inputs)) * noise_std  # add noise to input
    d = {}  # to store the pre-activation, activation, mean and variance for each layer
    # The data for labeled and unlabeled examples are stored separately
    d['labeled'] = {'z': {}, 'm': {}, 'v': {}, 'h': {}}
    d['unlabeled'] = {'z': {}, 'm': {}, 'v': {}, 'h': {}}
    d['labeled']['z'][0], d['unlabeled']['z'][0] = split_lu(h)
    for l in range(1, L+1):
        print ("Layer ", l, ": ", layer_sizes[l-1], " -> ", layer_sizes[l])
        d['labeled']['h'][l-1], d['unlabeled']['h'][l-1] = split_lu(h)
        z_pre = tf.matmul(h, weights['W'][l-1])  # pre-activation
        z_pre_l, z_pre_u = split_lu(z_pre)  # split labeled and unlabeled examples

        m, v = tf.nn.moments(z_pre_u, axes=[0])

        # if training:
        def training_batch_norm():
            # Training batch normalization
            # batch normalization for labeled and unlabeled examples is performed separately
            if noise_std > 0:
                # Corrupted encoder
                # batch normalization + noise
                z = join(batch_normalization(z_pre_l), batch_normalization(z_pre_u, m, v))
                z += tf.random_normal(tf.shape(z_pre)) * noise_std
            else:
                # Clean encoder
                # batch normalization + update the average mean and variance using batch mean and variance of labeled examples
                z = join(update_batch_normalization(z_pre_l, l), batch_normalization(z_pre_u, m, v))
            return z

        # else:
        def eval_batch_norm():
            # Evaluation batch normalization
            # obtain average mean and variance and use it to normalize the batch
            mean = ewma.average(running_mean[l-1])
            var = ewma.average(running_var[l-1])
            z = batch_normalization(z_pre, mean, var)
            # Instead of the above statement, the use of the following 2 statements containing a typo
            # consistently produces a 0.2% higher accuracy for unclear reasons.
            return z

        # perform batch normalization according to value of boolean "training" placeholder:
        z = tf.cond(training, training_batch_norm, eval_batch_norm)

        if l == L:
            # use softmax activation in output layer
            h = tf.nn.softmax(weights['gamma'][l-1] * (z + weights["beta"][l-1]))
        else:
            # use ReLU activation in hidden layers
            h = tf.nn.relu(z + weights["beta"][l-1])
        d['labeled']['z'][l], d['unlabeled']['z'][l] = split_lu(z)
        d['unlabeled']['m'][l], d['unlabeled']['v'][l] = m, v  # save mean and variance of unlabeled examples for decoding
    d['labeled']['h'][l], d['unlabeled']['h'][l] = split_lu(h)
    return h, d

print ("=== Corrupted Encoder ===")
y_c, corr = encoder(inputs, noise_std)

print ("=== Clean Encoder ===")
y, clean = encoder(inputs, 0.0)  # 0.0 -> do not add noise

print ("=== Decoder ===")


def g_gauss(z_c, u, size):
    "gaussian denoising function proposed in the original paper"
    wi = lambda inits, name: tf.Variable(inits * tf.ones([size]), name=name)
    a1 = wi(0., 'a1')
    a2 = wi(1., 'a2')
    a3 = wi(0., 'a3')
    a4 = wi(0., 'a4')
    a5 = wi(0., 'a5')

    a6 = wi(0., 'a6')
    a7 = wi(1., 'a7')
    a8 = wi(0., 'a8')
    a9 = wi(0., 'a9')
    a10 = wi(0., 'a10')

    mu = a1 * tf.sigmoid(a2 * u + a3) + a4 * u + a5
    v = a6 * tf.sigmoid(a7 * u + a8) + a9 * u + a10

    z_est = (z_c - mu) * v + mu
    return z_est

# Decoder
z_est = {}
d_cost = []  # to store the denoising cost of all layers
for l in range(L, -1, -1):
    print ("Layer ", l, ": ", layer_sizes[l+1] if l+1 < len(layer_sizes) else None, " -> ", layer_sizes[l], ", denoising cost: ", denoising_cost[l])
    z, z_c = clean['unlabeled']['z'][l], corr['unlabeled']['z'][l]
    m, v = clean['unlabeled']['m'].get(l, 0), clean['unlabeled']['v'].get(l, 1-1e-10)
    if l == L:
        u = unlabeled(y_c)
    else:
        u = tf.matmul(z_est[l+1], weights['V'][l])
    u = batch_normalization(u)
    z_est[l] = g_gauss(z_c, u, layer_sizes[l])
    z_est_bn = (z_est[l] - m) / v
    # append the cost of this layer to d_cost
    d_cost.append((tf.reduce_mean(tf.reduce_sum(tf.square(z_est_bn - z), 1)) / layer_sizes[l]) * denoising_cost[l])

# calculate total unsupervised cost by adding the denoising cost of all layers
u_cost = tf.add_n(d_cost)

y_N = labeled(y_c)
cost = -tf.reduce_mean(tf.reduce_sum(outputs*tf.log(y_N), 1))  # supervised cost
loss = cost + u_cost  # total cost

pred_cost = -tf.reduce_mean(tf.reduce_sum(outputs*tf.log(y), 1))  # cost used for prediction
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(outputs, 1))  # no of correct predictions
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) * tf.constant(100.0)
    tf.summary.scalar("accuracy", accuracy)

#learning_rate = tf.Variable(starter_learning_rate, trainable=False)
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    


# add the updates of batch normalization statistics to train_step
bn_updates = tf.group(*bn_assigns)
with tf.control_dependencies([train_step]):
    train_step = tf.group(bn_updates)

print ("===  Loading Data ===")

#load the data
pickle_file = "./gmql_data/Datasets/breast_data_all_divided_4.pickle"
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    del save  # hint to help gc free up memory
class DataSets(object):
    pass
data_sets = DataSets()

train_labels = dense_to_one_hot(fix_label(train_labels)).astype(numpy.float32)
test_labels = dense_to_one_hot(fix_label(test_labels)).astype(numpy.float32)
valid_labels = dense_to_one_hot(fix_label(valid_labels)).astype(numpy.float32)

#combine train and test set for cross validation
newtrainlabel = numpy.concatenate((train_labels, test_labels), axis=0)
newtraindata = numpy.concatenate((train_dataset, test_dataset), axis=0)

x = newtraindata
z = newtrainlabel

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
sss.get_n_splits(x, z)

# 5X cross validation
for train_index, valid_index in sss.split(x, z):
    
    x_train, x_valid = x[train_index], x[valid_index]
    z_train, z_valid = z[train_index], z[valid_index]
    #print (z_train )
    
    data_sets.train = SemiDataSet(x_train,z_train , 60)
    data_sets.validation = DataSet(valid_dataset,valid_labels)
    data_sets.test = DataSet(x_valid,z_valid )
    
    print (data_sets.train.unlabeled_ds.labels.shape)
    print (data_sets.train.unlabeled_ds.dataset.shape)
    expression_dataset = data_sets

    saver = tf.train.Saver(write_version=tf.train.SaverDef.V1)



    print ("===  Starting Session ===")

    sess = tf.Session()
    print("session started")
    i_iter = 0
    #create checkpoint 
#     ckpt = tf.train.get_checkpoint_state('checkpoints_cancer/')  # get latest checkpoint (if any)
#     if ckpt and ckpt.model_checkpoint_path:
#         # if checkpoint exists, restore the parameters and set epoch_n and i_iter
#         saver.restore(sess, ckpt.model_checkpoint_path)
#         epoch_n = int(ckpt.model_checkpoint_path.split('-')[1])
#         i_iter = (epoch_n+1) * (num_examples//batch_size)
#         print ("Restored Epoch ", epoch_n)
#     else:
#         # no checkpoint exists. create checkpoints directory if it does not exist.
#         if not os.path.exists('checkpoints_cancer'):
#             os.makedirs('checkpoints_cancer')
#         writer = tf.summary.FileWriter('./log', sess.graph)
#         init = tf.global_variables_initializer()
#         sess.run(init)
    

    init = tf.global_variables_initializer()
    sess.run(init)




    print ("=== Training ===")
    print ("Initial Accuracy: ", sess.run(accuracy, feed_dict={inputs: expression_dataset.test.dataset, outputs: expression_dataset.test.labels, training: False}), "%")


    for i in tqdm(range(i_iter, num_iter)):
        dataset, labels = expression_dataset.train.next_batch(batch_size)
        sess.run(train_step, feed_dict={inputs: dataset, outputs: labels, training: True})

        if (i > 1) and ((i+1) % (num_iter//num_epochs) == 0):
            epoch_n = i//(num_examples//batch_size)
            sess.run(accuracy, feed_dict={inputs: expression_dataset.test.dataset, outputs:expression_dataset.test.labels, training: False})

            print ("Epoch ", epoch_n, ", Accuracy: ", sess.run(accuracy, feed_dict={inputs: expression_dataset.test.dataset, outputs:expression_dataset.test.labels, training: False}), "%")     
            with open('train_log_cancer', 'a') as train_log:
                # write test accuracy to file "train_log"
                train_log_w = csv.writer(train_log)
                log_i = [epoch_n] + sess.run([accuracy], feed_dict={inputs: expression_dataset.test.dataset, outputs: expression_dataset.test.labels, training: False})
                train_log_w.writerow(log_i)
            
            if sess.run(accuracy, feed_dict={inputs: expression_dataset.test.dataset, outputs:expression_dataset.test.labels, training: False})> 99 and epoch_n > 30:
                break



    print ("Final Accuracy: ", sess.run(accuracy, feed_dict={inputs: expression_dataset.validation.dataset, outputs: expression_dataset.validation.labels, training: False}), "%")
    with open('train_log_cancer', 'a') as train_log:
    # write test accuracy to file "train_log"
      train_log_w = csv.writer(train_log)
      log_i = ["Final accuracy: "] + sess.run([accuracy], feed_dict={inputs: expression_dataset.validation.dataset, outputs: expression_dataset.validation.labels, training: False})
      train_log_w.writerow(log_i)
      train_log_w.writerow(["breast tum featurelar cv 5 "] )             

    print ("Final Accuracy: ", sess.run(accuracy, feed_dict={inputs: expression_dataset.validation.dataset, outputs: expression_dataset.validation.labels, training: False}), "%")

    y_p = tf.argmax(y, 1)
    val_accuracy, y_pred = sess.run([accuracy,y_p], feed_dict={inputs: expression_dataset.validation.dataset, outputs: expression_dataset.validation.labels, training: False})
    print ("validation accuracy:", val_accuracy)
    y_true = numpy.argmax(expression_dataset.validation.labels,1)
    print ("Precision", sk.metrics.precision_score(y_true, y_pred))
    print ("Recall", sk.metrics.recall_score(y_true, y_pred))
    print ("f1_score", sk.metrics.f1_score(y_true, y_pred))
    print ("confusion_matrix")
    print (sk.metrics.confusion_matrix(y_true, y_pred))


    sess.close()

=== Corrupted Encoder ===
Layer  1 :  20207  ->  2000
Layer  2 :  2000  ->  1000
Layer  3 :  1000  ->  500
Layer  4 :  500  ->  250
Layer  5 :  250  ->  10
Layer  6 :  10  ->  2
=== Clean Encoder ===
Layer  1 :  20207  ->  2000
Layer  2 :  2000  ->  1000
Layer  3 :  1000  ->  500
Layer  4 :  500  ->  250
Layer  5 :  250  ->  10
Layer  6 :  10  ->  2
=== Decoder ===
Layer  6 :  None  ->  2 , denoising cost:  0.1
Layer  5 :  2  ->  10 , denoising cost:  0.1
Layer  4 :  10  ->  250 , denoising cost:  0.1
Layer  3 :  250  ->  500 , denoising cost:  0.1
Layer  2 :  500  ->  1000 , denoising cost:  0.1
Layer  1 :  1000  ->  2000 , denoising cost:  10.0
Layer  0 :  2000  ->  20207 , denoising cost:  1000.0
===  Loading Data ===
730
244
244
(730, 2)
(730, 20207)
===  Starting Session ===
session started
=== Training ===
Initial Accuracy:  7.78688 %


  1%|██                                                                                                                                                                                                                           | 11/1200 [00:02<03:24,  5.82it/s]

Epoch  0 , Accuracy:  8.19672 %


  2%|████▏                                                                                                                                                                                                                        | 23/1200 [00:04<03:00,  6.52it/s]

Epoch  1 , Accuracy:  74.5902 %


...

Epoch  98 , Accuracy:  97.9508 %


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 1199/1200 [03:26<00:00,  6.49it/s]

Epoch  99 , Accuracy:  97.9508 %


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1200/1200 [03:26<00:00,  4.88it/s]


Final Accuracy:  98.3606 %
Final Accuracy:  98.3607 %
validation accuracy: 98.3606
Precision 0.990950226244
Recall 0.990950226244
f1_score 0.990950226244
confusion_matrix
[[ 21   2]
 [  2 219]]
